In [1]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))

import torch
import torch.nn.functional as F
from torch import nn

import safetensors
from accelerate import notebook_launcher

import einops

from utils.data_utils import BrainDataset, get_tokenizer
from utils.train_utils import TrainConfig, run_train_model, count_parameters, load_model_weights

from transformers import GPT2Tokenizer

from models.bert import BrainBert, BertConfig
from models.vq_brain_per_channel import SoundStream, VAEConfig

from dataclasses import dataclass
from simple_parsing.helpers import Serializable

from safetensors.torch import load_model
import albumentations as A


In [2]:
# device='cuda'
vae_config = VAEConfig(C=256, levels=(8, 8, 6, 5))
vq_vae = SoundStream(**vae_config.to_dict())

vq_weights = "/drive/logs/kovalev/vq_brain/medium_14M_256ws_8x_2000/step_78000_loss_0.0275.safetensors"
vq_vae = load_model_weights(vq_vae, vq_weights)

self.codebook_size 1920
self.downsample 8
load compiled weights


In [3]:
model_config = BertConfig(dim=256, 
                 window_size=512, 
                 tokenizer_downsample=int(vq_vae.downsample),
                 n_electrodes=256, 
                 mask_ratio=0.5, 
                 n_layers=12, 
                 n_heads=12, 
                 n_kv_heads=12)

model = BrainBert(model_config, vq_vae)

for param in model.tokenizer.parameters():
    param.requires_grad = False 

count_parameters(model)

BertConfig(window_size=512, n_electrodes=256, mask_ratio=0.5, tokenizer_downsample=8, n_layers=12, dim=256, hidden_dim=1024, head_dim=32, n_heads=12, n_kv_heads=12)
Encoder: number of parameters: 42.79M
Total: 42.79M, Trainable: 29.39M


(42788997, 29391488)

In [4]:
# x = torch.randn(1, 512, 256)
# model(x)

### Run training pipeline

In [5]:
window_size = model_config.window_size
n_electrodes = 256

train_transform = A.Compose([
    A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
                  border_mode=4, value=0, always_apply=True),
    A.RandomCrop(height=window_size, width=n_electrodes, always_apply=True),
])

test_transform = A.Compose([
    A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
                  border_mode=4, value=0, always_apply=True),
    A.CenterCrop(height=window_size, width=n_electrodes, always_apply=True),
])

In [6]:
data_path = Path("/drive/data/competitionData")

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

train_dataset = BrainDataset(data_path / 'train', tokenize_function=get_tokenizer(tokenizer), transform=train_transform)
test_dataset = BrainDataset(data_path / 'test', tokenize_function=get_tokenizer(tokenizer), transform=test_transform)

Runed processing of the  /drive/data/competitionData/train


/opt/conda/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


len of the dataset: 8800
max input len 906
median len 297.0
Runed processing of the  /drive/data/competitionData/test
len of the dataset: 880
max input len 919
median len 283.5


In [ ]:
project_name = 'bert'
save_folder = Path("/drive/logs/kovalev")

train_config = TrainConfig(exp_name='new_vqvae_8x_2000_large_bert_30M',
                           mixed_precision=True,
                           batch_size=2, 
                           num_workers=3,
                           pin_memory=True, 
                           eval_interval=500, 
                           learning_rate=3e-4,
                           weight_decay=0, 
                           grad_clip=5,
                           lr_decay_iters=30_000, 
                           warmup_iters=1000, 
                           project_name=project_name, 
                           save_folder=save_folder
                          )

# model = torch.compile(model)
args = (model, (train_dataset, test_dataset), train_config, model_config)
notebook_launcher(run_train_model, args, num_processes=1)

Launching training on one GPU.


/opt/conda/envs/pytorch/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
wandb: Currently logged in as: koval_alvi. Use `wandb login --relogin` to force relogin


Device for training:  cuda
Num devices:  1
Completed initialization of scheduler
********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
overall_steps 500: 3.2756600379943848
val loss: 3.272568941116333
saved model:  step_500_loss_3.2726.safetensors
*****************************************************************************************************************************************************************************************************************************************************************************************************************

In [ ]:
save_folder = Path("/drive/logs/kovalev")

safetensors.torch.save_model(model, save_folder / 'model_mae_long_train.safetensors')

In [ ]:
model

In [ ]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [ ]:
model = model._orig_mod

In [ ]:
mae_weights = "/drive/logs/kovalev/fixed_abs_mae_11M_5M_spikes/step_34000_loss_0.0166.safetensors"

mae_model = SimpleMAE( SimpleEncoderConfig(), SimpleMAEConfig())
mae_model = torch.compile(mae_model)
safetensors.torch.load_model(mae_model, mae_weights)
mae_model = mae_model._orig_mod

brain_model = mae_model.encoder

In [ ]:
x = test_dataset[0][0]
device = 'cpu'
x = torch.from_numpy(x[None, :]).to(device)
print(x.shape)
loss, y, binary = model(x, masking_ratio=0.75,  return_preds=True)

x = x.detach().cpu()[0]
y = y.detach().cpu()[0]
binary = binary.detach().cpu()[0]

import matplotlib.pyplot as plt 

# plt.plot(y[1, :])
# plt.show()
# plt.plot(x[1, :])
# plt.show()
loss = F.l1_loss(y, x, reduction='none')
print(torch.mean(loss[loss>0]))

plt.show()
plt.imshow(binary[:64].T, aspect='auto')
# plt.colorbar()
plt.show()
plt.imshow(torch.clip(y[:64].T, 0, 1), aspect='auto')
plt.colorbar()
plt.show()
plt.imshow(x[:64].T, aspect='auto' )
plt.colorbar()


In [ ]:
plt.show()
plt.plot(binary[:32, 10])
plt.show()
plt.plot(y[:32, 10])
# plt.show()
plt.plot(x[:32, 10])